In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from PIL import Image
import os
import sys

sys.path.insert(0, "..")
from preprocess import VietnameseTextCleaner
from utils import dict_handler
from transforms import ImageTransform


matplotlib.style.use("ggplot")

In [2]:
CUR_DIR = os.path.abspath(os.curdir)
VNCORE_NLP_PATH = os.path.join(CUR_DIR, "../vncorenlp/")
STOPWORDS_PATH = os.path.join(CUR_DIR, "../stop_words/vietnamese-stopwords-dash.txt")

cleaner = VietnameseTextCleaner(
    stopwords_path=STOPWORDS_PATH,
    vncorenlp_path=VNCORE_NLP_PATH,
    cur_dir=CUR_DIR,
)

In [4]:
df = pd.read_csv(r"C:\Users\anhbu\Desktop\new_odl\data\reintel2020\private_test.csv")

In [9]:
df = df[["id", "post_message"]]
df.post_message = df.post_message.apply(cleaner.clean_one)

In [11]:
id = df.id.values
post_message = df.post_message.values

In [ ]:
import torch

torch.save(
    {"id": id, "post_message": post_message},
    "private_test_cleaned.pt",
)

In [1]:
import sys
import torch

sys.path.insert(0, "..")
from module import TextCNN, BertEmbedding
import torch.nn as nn


MAX_LENGTH = 65
EMBEDDING_DIM = 768
NUM_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
LR = 1e-5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

G = BertEmbedding(max_length=MAX_LENGTH, device=device).to(device)
F = TextCNN(EMBEDDING_DIM, NUM_FILTERS, FILTER_SIZES).to(device)

cache = torch.load(
    "D:/storage/odl/checkpoints/offline/reintel2020/TextCNN_BERT/checkpoint.pt"
)
F.load_state_dict(cache["model_F"])
G.load_state_dict(cache["model_G"])

F.eval()
G.eval()

BertEmbedding(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [2]:
import torch
import numpy

test = torch.load("./private_test_cleaned.pt")
id = test["id"]
post_message = test["post_message"]

label = []

for batch in numpy.array_split(numpy.array(post_message), 50):
    embbed = G(batch)
    label += torch.softmax(F(embbed), dim=1)[:, 1].tolist()

import pandas as pd

df = pd.DataFrame({"id": id, "label": label})
df.to_csv("./results.csv", index=False, header=False)